In [1]:
import psycopg2
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType
from textblob import TextBlob
import os

# Fonction pour analyser le sentiment
def analyze_sentiment(content):
    if content:
        analysis = TextBlob(content)
        if analysis.sentiment.polarity > 0:
            return "positif"
        elif analysis.sentiment.polarity < 0:
            return "négatif"
        else:
            return "neutre"
    return "neutre"

# 1. Création d'une session Spark
spark = SparkSession.builder \
    .appName("MastodonDataProcessing") \
    .getOrCreate()

# 2. Connexion à la base de données PostgreSQL
jdbc_url = "jdbc:postgresql://postgres:5432/" + os.getenv('DB_NAME')
properties = {
    "user": os.getenv('DB_USER'),
    "password": os.getenv('DB_PASSWORD'),
    "driver": "org.postgresql.Driver"
}

# Connexion pour exécuter des commandes SQL
conn = None
try:
    conn = psycopg2.connect(
        dbname=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        host="postgres",
        port="5432"
    )
    cursor = conn.cursor()

    # 3. Création de la table gold_mastodon si elle n'existe pas
    create_table_query = """
    CREATE TABLE IF NOT EXISTS gold_mastodon (
        post_id BIGINT PRIMARY KEY,
        username VARCHAR(255),
        content TEXT,
        total_engagement INTEGER,
        created_at TIMESTAMP,
        hashtags TEXT,
        sentiment_category VARCHAR(50),
        average_engagement_per_post DOUBLE PRECISION
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    # 4. Lire les données depuis la table silver_mastodon
    posts_df = spark.read.jdbc(url=jdbc_url, table="silver_mastodon", properties=properties)

    # 5. Compter le nombre de lignes lues dans la table silver_mastodon
    silver_count = posts_df.count()

    # 6. Calculer les statistiques nécessaires
    aggregated_df = posts_df.groupBy("id", "username", "cleaned_content", "created_at") \
        .agg(
            (F.sum("favourites_count") + F.sum("reblogs_count") + F.sum("replies_count")).alias("total_engagement"),
            F.concat_ws(", ", F.collect_list("hashtags")).alias("hashtags"),  # Transformation en texte brut
            F.avg("engagement_count").alias("average_engagement_per_post")
        )

    # 7. Renommer les colonnes
    aggregated_df = aggregated_df.withColumnRenamed("cleaned_content", "content") \
        .withColumnRenamed("id", "post_id")

    # 8. Lire les données existantes dans gold_mastodon
    existing_gold_df = spark.read.jdbc(url=jdbc_url, table="gold_mastodon", properties=properties)

    # 9. Identifier les posts à mettre à jour
    posts_to_update_df = aggregated_df.join(existing_gold_df, on="post_id", how="inner")

    # 10. Identifier les nouveaux posts à insérer
    posts_to_insert_df = aggregated_df.join(existing_gold_df, on="post_id", how="left_anti")

    # Analyser le sentiment pour les nouveaux posts
    sentiment_analysis_udf = F.udf(analyze_sentiment, StringType())
    posts_to_insert_df = posts_to_insert_df.withColumn("sentiment_category", sentiment_analysis_udf(F.col("content")))

    # Compter les nouveaux posts à insérer
    new_posts_count = posts_to_insert_df.count()

    # 12. Mettre à jour les posts existants
    updated_count = 0  # Compteur pour les mises à jour
    if posts_to_update_df.count() > 0:
        update_query = """
        UPDATE gold_mastodon SET 
            total_engagement = %s, 
            average_engagement_per_post = %s,
            sentiment_category = %s
        WHERE post_id = %s
        """
        for row in posts_to_update_df.collect():
            cursor.execute(update_query, (
                row.total_engagement, 
                row.average_engagement_per_post,
                row.sentiment_category,
                row.post_id
            ))
            updated_count += 1  # Incrémenter le compteur
        conn.commit()

    # Insérer les nouveaux posts
    if new_posts_count > 0:
        posts_to_insert_df.write.jdbc(url=jdbc_url, table="gold_mastodon", mode="append", properties=properties)

    # 14. Afficher les données mises à jour
    gold_data_updated_df = spark.read.jdbc(url=jdbc_url, table="gold_mastodon", properties=properties)
    
    # Compter le nombre total de posts dans gold_mastodon après les opérations
    total_gold_count = gold_data_updated_df.count()

    # Impressions finales
    print(f"Nombre de lignes lues dans 'silver_mastodon': {silver_count}")
    print(f"Nombre de posts mis à jour dans 'gold_mastodon': {updated_count}")
    print(f"Nombre de nouveaux posts insérés dans 'gold_mastodon': {new_posts_count}")  
    print(f"Nombre total de posts dans 'gold_mastodon': {total_gold_count}")

finally:
    if conn:
        cursor.close()
        conn.close()

# Arrêter la session Spark
spark.stop()

Nombre de lignes lues dans 'silver_mastodon': 268
Nombre de posts mis à jour dans 'gold_mastodon': 0
Nombre de nouveaux posts insérés dans 'gold_mastodon': 268
Nombre total de posts dans 'gold_mastodon': 268
